In [1]:
import numpy as np
import cv2
import pandas as pd

import matplotlib.pyplot as plt
from glob import glob

In [2]:
# 根据evaluate的曲线位置（0-11）和evaluate的range（1,2,3意味着几倍）生成取样区域
def eva_area(img, pos=1, rang=1):
    
    num = pos//3
    loc = pos%3
    center = 130+300*num
    pic = np.zeros([630, 1165])# 这里为了calibrate改了宽度
    if loc == 2:
        R = 25
    elif loc == 1:
        R = 50
    elif loc == 0:
        R = 100
    
    thick = np.round(rang*6.5).astype(int)
    cv2.circle(pic, (center,130), R, 255, thick)
    cv2.rectangle(pic, (center-R-thick,130), (center+R+thick,130+R+thick), 0, -1)
    cv2.line(pic, (center-R,130), (center-R,130+470), 255, thick) 
    cv2.line(pic, (center+R,130), (center+R,130+470), 255, thick) 
    
    if img.shape[1]/img.shape[0] != 116/63:
        print('Image shape is not defult:', img.shape)
    pic = np.round(cv2.resize(pic, (img.shape[1],img.shape[0]))/255.)*255
    
    return pic

In [3]:
# 输入名字 输出pos, x, y
def get_numbers(traj_name):
    pred = cv2.imread('binary_trajectorys\\' + traj_name, 0)
    height, width = pred.shape[0],pred.shape[1]
    
    n1 = int(traj_name[-6])
    n2 = traj_name[-5]
    if n2 == 'o':
        n2 = 0
    elif n2 == 'm':
        n2 = 1
    elif n2 == 'i':
        n2 = 2
    pos = (n1-1)*3+n2
    x = 130+300*(pos//3)
    y = int(13/63*height)
    
    return pos, x, y

In [18]:
# 输入一个trajectory图片和一个GT图片 输出inversed precision
def calc_ipres(GT, pred):
    GT = np.round(GT/255.).astype(int)
    pred = np.round(pred/255.).astype(int)
    tot = np.count_nonzero(pred)
    TP = np.count_nonzero(GT&pred)

    if tot > 0:
        return 1-TP/tot
    else:
#         print('Print is too bad, please ditch this trajectory.')
        return 1

In [11]:
# 计算某个trajectory arc的AIP
# 输入12个叠在一起的trajectory图片，输出从range1-5曲线下的面积
def get_AP_section(traj_name):
    AP = 0
    pred = cv2.imread('binary_trajectorys\\' + traj_name, 0)
    pos, x, y = get_numbers(traj_name)
    
    height, width = pred.shape
    for i in range(5):
        GT = eva_area(pred, pos, i+1)
        pred_s = pred[:y, :]
        GT_s = GT[:y, :]

        cv2.imwrite('GTtest.png', GT_s)
        cv2.imwrite('Predtest.png', pred_s)
        Pres = calc_ipres(GT_s, pred_s)
        AP += Pres*0.2

    return AP

In [38]:
# 可视化单个trajectory AIP并分割ARC区域，打上AIP score
def view_AP(traj_name, rang=1):
    pred = cv2.imread('binary_trajectorys\\' + traj_name, 0)
    height, width = pred.shape[0],pred.shape[1]
    blk = np.zeros([height, width])
    text_blk = np.zeros([width, height,3])
    pos, x, y = get_numbers(traj_name)
    
    GT = eva_area(pred, pos, rang)
    pred = pred.reshape([height, width,1])
    GT = GT.reshape([height, width,1])
    blk = blk.reshape([height, width,1])
    
    AP_arc = get_AP_section(traj_name)

    overlay = np.concatenate([blk, pred, GT], axis=-1)
    cv2.line(overlay, (0, y), (width, y), (255,255,255), 1) 

    cv2.putText(overlay, 'Arc FML: ' + str(round(AP_arc, 2)), (x-70, 20),  cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (255,255,255), 1, cv2.LINE_AA)

    return AP_arc, overlay 

In [39]:
# 图也保存数据保存excel
names = []
APs_arc = []
files = glob('binary_trajectorys\\*.*')
for file in files:
    name = list(file.split('\\'))[-1]
    names.append(name)
    AP_arc, overlay = view_AP(name, rang=1)
    APs_arc.append(AP_arc)
    cv2.imwrite('AIP_results\\' + name, overlay)
    
# df = pd.DataFrame({'Pic_name': names, 'ArcAIP': APs_arc})
# df.to_excel('AIP summary.xlsx')

In [35]:
# 可视化单个trajectory AIP并分割ARC区域，打上AIP score
def view_range_AP(traj_name, rang=1):
    pred = cv2.imread('binary_trajectorys\\' + traj_name, 0)
    height, width = pred.shape[0],pred.shape[1]
    blk = np.zeros([height, width])
    text_blk = np.zeros([width, height,3])
    pos, x, y = get_numbers(traj_name)
    
    GT = eva_area(pred, pos, rang)
    pred = pred.reshape([height, width,1])
    GT = GT.reshape([height, width,1])
    blk = blk.reshape([height, width,1])
    
    AP_arc = calc_ipres(GT, pred)
    
    overlay = np.concatenate([blk, pred, GT], axis=-1)
    cv2.line(overlay, (0, y), (width, y), (255,255,255), 1) 

    cv2.putText(overlay, 'Arc FML: ' + str(round(AP_arc, 2)), (x-70, 20), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (255,255,255), 1, cv2.LINE_AA)
    cv2.putText(overlay, 'Range=' + str(rang), (x-50, 100), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (255,255,255), 1, cv2.LINE_AA)

    return AP_arc, overlay 

In [37]:
for r in range(5):
    AP_arc, overlay = view_range_AP('croped_test13,14,15,16-v2-1o.bmp', rang=r+1)
    print(AP_arc)
    cv2.imwrite('FMLdemo_' + str(r+1) + '.png', overlay)

0.4444306623666584
0.03187794591912674
0.0002480774001488717
0.0
0.0


In [73]:
# input traj_name, output group name, version and R[innner5/medium10/outer20]
def get_group_v_R(traj_name):
    n1 = int(traj_name[-6])
    n2 = traj_name[-5]
    if n2 == 'o':
        R = 20
    elif n2 == 'm':
        R = 10
    elif n2 == 'i':
        R = 5
        
    group_set, version = list(traj_name.split('-'))[0], list(traj_name.split('-'))[1]
    group_set = list(group_set.split('test'))[-1]
    group = list(group_set.split(','))[n1-1]
    
    return int(group), version, R

In [74]:
# input group name, output printing temperature, printing speed, fibre feed rate and nylon feed rate
# def get_paras(group):
#     params = pd.read_excel('AIIoU summary.xlsx', sheet_name='Sheet2')
#     p_tem = params[params['No.'] == group]['Printing Temperature'].to_numpy()[0]
#     p_speed = params[params['No.'] == group]['Printing Speed'].to_numpy()[0]
#     f_rate = params[params['No.'] == group]['Fibre Feed rate'].to_numpy()[0]
#     n_rate = params[params['No.'] == group]['Nylon Feed rate'].to_numpy()[0]
#     return p_tem, p_speed, f_rate, n_rate

In [76]:
# 根据组输出对应组合的参数
# df = pd.read_excel('AIIoU summary.xlsx', sheet_name='Sheet2')
def get_paras(group):
    df = pd.read_excel('AIIoU summary.xlsx', sheet_name='Sheet2')
    return df[df['No.']==group].to_numpy()[0]

In [86]:
# input AIIoU summary excel, output df {printing temperature, printing speed, fibre feed rate, nylon feed rate, R, version, APs}
def rearragen_AIIoU_excel(excel_name):
    df = pd.read_excel(excel_name, sheet_name='Sheet1')
    versions = []
    Rs = []
    p_tems = []
    p_speeds = []
    f_rates = []
    n_rates = []
    names = list(df['Pic_name'])
#     AP_befores = list(df['AP_before'])
#     AP_arcs = list(df['AP_arc'])
#     AP_afters = list(df['AP_after'])
    for name in names:
        group, version, R = get_group_v_R(name)
        versions.append(version)
        Rs.append(R)
        _, p_tem, p_speed, f_rate, n_rate = get_paras(group)
        p_tems.append(p_tem)
        p_speeds.append(p_speed)
        f_rates.append(f_rate)
        n_rates.append(n_rate)
    scores = list(df['ArcFML'])

    new_df = pd.DataFrame({'Printing Temperature': p_tems, 'Arc Printing Speed': p_speeds, 'Arc Fibre Feed Rate': f_rates, 
                           'Arc Nylon Feed Rate': n_rates, 'Arc Radius': Rs, 'Version': versions, 
                           'Score': scores})
    return new_df
df = rearragen_AIIoU_excel('FML summary.xlsx')

In [87]:
df.head()

,Printing Temperature,Arc Printing Speed,Arc Fibre Feed Rate,Arc Nylon Feed Rate,Arc Radius,Version,Score
0,270.0,120.0,1.1,0.8,20,v2,0.070766
1,270.0,120.0,1.1,0.8,20,v3,0.073548
2,260.0,280.0,1.1,1.0,20,v1,0.074956
3,250.0,440.0,1.1,0.6,20,v3,0.076316
4,270.0,120.0,1.1,0.8,20,v1,0.078207


In [79]:
# Taguchi S/N calculation in FML
def get_SN(ys):
    MSD = np.sum(np.array(ys)**2)/len(ys)
    return -10*np.log10(MSD)

In [82]:
# 根据组数和radius抓取所有version计算SN ratio
def get_version_array_at_R(group, r):
    _, pt, ps, ffr, nfr = get_paras(group)
    v_array = df[(df['Printing Temperature']==int(pt))& \
               (df['Arc Printing Speed']==int(ps)) & (df['Arc Fibre Feed Rate']==ffr) & \
               (df['Arc Nylon Feed Rate']==nfr) & (df['Arc Radius']==r)]['Score'].to_numpy()

    return get_SN(v_array)

In [83]:
# 根据组数所有version计算SN ratio
def get_version_array(group):
#     print(get_paras(group))
    _, pt, ps, ffr, nfr = get_paras(group)
    v_array = df[(df['Printing Temperature']==pt)& \
               (df['Arc Printing Speed']==ps) & (df['Arc Fibre Feed Rate']==ffr) & \
               (df['Arc Nylon Feed Rate']==nfr)]['Score'].to_numpy()
    return get_SN(v_array)

In [107]:
# total的SN ratio
array = np.zeros([16,6])
for g in range(16):
    array[g,:5] = get_paras(g+1)
    array[g,5] = get_version_array(g+1)
#     array[g,6] = get_version_array_at_R(g+1, 10)
#     array[g,7] = get_version_array_at_R(g+1, 20)
new_df = pd.DataFrame({'Group': array[:,0], 'Printing Temperature': array[:,1], 'Printing Speed': array[:,2], 'Fibre Feed Rate': array[:,3], 'Nylon Feed Rate': array[:,4]\
                  , 'SN ratio': array[:,5]})
# new_df.to_excel('FML Total SN Ratio.xlsx')

In [101]:
# 计算每组的r分别为5，10，20的sn ratio并集结成excel
array = np.zeros([16,8])
for g in range(16):
    array[g,:5] = get_paras(g+1)
    array[g,5] = get_version_array_at_R(g+1, 5)
    array[g,6] = get_version_array_at_R(g+1, 10)
    array[g,7] = get_version_array_at_R(g+1, 20)
new_df = pd.DataFrame({'Group': array[:,0], 'Printing Temperature': array[:,1], 'Printing Speed': array[:,2], 'Fibre Feed Rate': array[:,3], 'Nylon Feed Rate': array[:,4]\
                  , 'Radius 2.5': array[:,5], 'Radius 5': array[:,6], 'Radius 10': array[:,7]})
# new_df.to_excel('FML SN Ratio.xlsx')

In [89]:
PTs = [240,250,260,270]
APSs = [120,280,440,600]
AFFRs = [0.8,0.9,1,1.1]
ANFRs = [0.6,0.8,1,1.2]

In [111]:
factor = 'Fibre Feed Rate'
# R = 'Radius 2.5'

if factor == 'Printing Temperature':
    lvs = PTs
elif factor == 'Printing Speed':
    lvs = APSs
elif factor == 'Fibre Feed Rate':
    lvs = AFFRs
elif factor == 'Nylon Feed Rate':
    lvs = ANFRs
else:
    print('Check input factor')

for lv in lvs:
    print(factor, lv, new_df[new_df[factor]==lv]['SN ratio'].mean())#R
#     print('Radius %s S/N ratio of %s at %s equal %s is %s' %(R, section, factor, lv, SN))

Fibre Feed Rate 0.8 2.5954620066216885
Fibre Feed Rate 0.9 3.134361334394229
Fibre Feed Rate 1 6.023633290288844
Fibre Feed Rate 1.1 12.12004853104737
